## **Are weeks of a subject similar in terms of mean nutritional intake ?**

In [1]:
import pandas as pd
import pingouin as pg
import warnings
import pickle
warnings.filterwarnings('ignore')


In [2]:
def save_dict_list_as(dic, file_name):
    with open(file_name, 'wb') as file:
        pickle.dump(dic, file)
    return 0
    

def load_dict_list(file_name):
    with open(file_name, 'rb') as file:
        loaded_dict = pickle.load(file)
    return loaded_dict

def scientific_notation(value):
    if abs(value) < 1 and value != 0:
        return '{:.2e}'.format(value)
    else:
        return '{:.2f}'.format(float(value))
    
week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
features = load_dict_list("Food_group/features.pkl")

## **Comparing mean intake btw week using ICC**

**Subject with at least 2 weeks (63) ==> comparing week1 with week2**

In [3]:
data = pd.read_csv('data_set\subject_full_week.csv')
mean = data.groupby(["subject_key", "week_of_year"])[features].mean().reset_index()

def ICC_week_comp(mean, nb_weeks) : 
    icc_scores_week ={}
    df = mean.copy()
    df = df.groupby("subject_key").head(nb_weeks)
    df["week_of_year"] = df.groupby('subject_key').cumcount() + 1
    for f in features:
        val = df[["subject_key", "week_of_year", f]]
        icc = pg.intraclass_corr(data=val, targets='subject_key', raters="week_of_year", ratings=f)
        score = icc.loc[icc['Type'] == 'ICC2k']["ICC"].values[0]
        icc_scores_week[f] = score

    mean_icc = pd.DataFrame(icc_scores_week, index=["ICC_mean"]).T
    return mean_icc


mean_icc = ICC_week_comp(mean, 2)
print(f"AVG ICC score, across all nutritional features, comparing weeks in terms of mean : {mean_icc.mean().values[0]}")
mean_icc




AVG ICC score, across all nutritional features, comparing weeks in terms of mean : 0.7328884849944076


,ICC_mean
eaten_quantity_in_gram,0.946487
energy_kcal_eaten,0.678210
energy_kj_eaten,0.689953
carb_eaten,0.803004
fat_eaten,0.714409
protein_eaten,0.857123
fiber_eaten,0.899393
alcohol_eaten,0.794656
beta_carotene_eaten,0.737854
calcium_eaten,0.825656


**Subject with at least 3 weeks (22) ==> assesing reliability week1, week2, week 3 with ICC**

In [4]:
df = data.copy()
key = df.groupby("subject_key")["week_of_year"].nunique() > 2
df = df[df['subject_key'].isin(key[key].index)]
mean = df.groupby(["subject_key", "week_of_year"])[features].mean().reset_index()
mean_icc = ICC_week_comp(mean, 3)
print(f"AVG ICC score, across all nutritional features, comparing weeks in terms of mean : {mean_icc.mean().values[0]}")
mean_icc


AVG ICC score, across all nutritional features, comparing weeks in terms of mean : 0.775111254797516


,ICC_mean
eaten_quantity_in_gram,0.947006
energy_kcal_eaten,0.645887
energy_kj_eaten,0.635877
carb_eaten,0.841738
fat_eaten,0.716913
protein_eaten,0.904563
fiber_eaten,0.948819
alcohol_eaten,0.941886
beta_carotene_eaten,0.874331
calcium_eaten,0.887282
